In [ ]:
import torch as tn
import datetime
import numpy as np
try:
    import torchtt as tntt
except:
    print('Installing torchTT...')
    %pip install git+https://github.com/ion-g-ion/torchTT
    import torchtt as tntt

In [ ]:
N = [10,11,12,13,14]
Rt = [1,3,4,5,6,1]
Rx = [1,6,6,6,6,1]
target = tntt.randn(N,Rt).round(0)
func = lambda x: 0.5*(x-target).norm(True)

In [ ]:
x0 = tntt.randn(N,Rx)
x =x0.clone()
for i in range(20):
    # compute riemannian gradient using AD    
    gr = tntt.manifold.riemannian_gradient(x,func)
    
    #stepsize length
    alpha = 1.0
    
    # update step
    x = (x-alpha*gr).round(0,Rx)    
    print('Value ' , func(x).numpy())

As a comparison, conventional gradient descent with respect to the TT cores is performed:

In [ ]:
y = x0.detach().clone()

for i in range(1000):
    tntt.grad.watch(y)
    fval = func(y)
    deriv = tntt.grad.grad(fval,y)    
    alpha = 0.00001 # for stability
    y = tntt.TT([y.cores[i].detach()-alpha*deriv[i] for i in range(len(deriv))])
    if (i+1)%100 == 0: print(func(y))
    

### Manifold tensor completion

One other task where the manifold learning can be applied is tensor completion.
The goal for this problem is to reconstruct a tensor in the TT format given only a few entries (possible noisy).

In [ ]:
N = 25
target = tntt.randn([N]*4,[1,2,3,3,1])
Xs = tntt.meshgrid([tn.linspace(0,1,N, dtype = tn.float64)]*4)
target = Xs[0]+1+Xs[1]+Xs[2]+Xs[3]+Xs[0]*Xs[1]+Xs[1]*Xs[2]+tntt.TT(tn.sin(Xs[0].full()))
target = target.round(1e-10)
print(target.R)

M = 15000 # number of observations 
indices = tn.randint(0,N,(M,4))

# observations are considered to be noisy
sigma_noise = 0.00001
obs = tn.normal(target.apply_mask(indices), sigma_noise)

# define the loss function
loss = lambda x: (x.apply_mask(indices)-obs).norm()**2

#%% Manifold learning
print('Riemannian gradient descent\n')
# starting point
x = tntt.randn([N]*4,[1,4,4,4,1])

tme = datetime.datetime.now()
# iterations
for i in range(10000):
    # manifold gradient 
    gr = tntt.manifold.riemannian_gradient(x,loss)

    step_size = 1.0
    R = x.R
    # step update
    x = (x - step_size * gr).round(0,R)

    # compute loss value
    if (i+1)%100 == 0:
        loss_value = loss(x)
        print('Iteration %4d loss value %e error %e tensor norm %e'%(i+1,loss_value.numpy(),(x-target).norm()/target.norm(), x.norm()**2))

tme = datetime.datetime.now() - tme
print('')
print('Time elapsed',tme)
print('Number of observations %d, tensor shape %s, percentage of entries observed %6.4f'%(M,str(x.N),100*M/np.prod(x.N)))
print('Number of unknowns %d, number of observations %d, DoF/observations %.6f'%(tntt.numel(x),M,tntt.numel(x)/M))

print('Rank after rounding',x.round(1e-6))